> ### 岭回归是对最小二乘法的优化

#### 首先最小二乘总是寻找拟合残差与特征正交的方向，但由于数据中存在噪声，按照最小二乘必须要满足正交的关系，可能会使得求解系数过大，导致模型过拟合。为了解决这个问题，岭回归加入对参数的惩罚项，通过L2范数和惩罚系数来约束参数的大小。

#### 同时也可以通过惩罚系数间接解决特征之间构成的矩阵不可逆问题

----
- #### 数学推导

#### $Loss = RSS + λω^Tω = (y - Xω)^T(y - Xω) + λω^Tω$
##### 化简后:
#### $Loss = y^Ty - y^TXω - ω^TX^Ty + ωX^TXω + λω^Tω$
##### 对$ω$求偏导:
#### $\frac{∂Loss}{∂ω} = -X^Ty - X^Ty + 2X^TXω + 2λω$
##### 令偏导为0，解得:
#### $ω = (X^TX + λI)^{-1}X^Ty$  (其中 $I$ 为单位矩阵) 

#### 对于惩罚系数 $λ$ 通常会使用交叉验证搜索得到，这里采用 $Loocv$ ，对于岭回归中 $Loocv$ 存在通解公式，以下为公式推导：

- ##### 以 $-i$ 为下标的变量表示去掉第 $i$ 个样本后的数据
- ##### $X$ 表示特征矩阵， $x_i$ 表示 $X$ 的第 $i$ 行向量的转置，$y$ 表示标签向量， $y_i$ 为该向量的第 $i$ 个元素（标量）


#### $step 0$
#### 简略描述一下 $Loocv$ 过程为：假定一个 $λ$ 后，将划分好的 $n$ 个样本集， $n$ 次分成 $n-1$ 个训练集，$1$ 个验证集，训练出 $n$ 模型， 算出平均模型残差以表示 $loss$ 评分， 即：
#### $loocv\_loss = \frac{1}{n}∑_{i = 1}^n || y_i - x_i^Tω_{-i,λ} ||^2$
#### $choose\_λ = min_{loocv\_loss}λ$

#### $step 1$
#### $ω_{-i,λ} = (X_{-i}^TX_{-i} + λI)^{-1}X^T_{-i}y_{-i}$
#### $ω_{-i,λ} = (X^TX - x_ix_i^T + λI)^{-1}(X^Ty - x_iy_i)$

- #### 为了进一步展开用到 $Sherman-Morrison$公式: $(A + uv^T)^{-1} = A^{-1} - \frac{A^{-1}uv^TA^{-1}}{1 + v^TA^{-1}u}$
- #### 令 $X^TX + λI = A$ 

#### $ω_{-i,λ} = \left(A^{-1}+\frac{A^{-1} x_{i} x_{i}^{T} A^{-1}}{1-x_{i}^{T} A^{-1} x_{i}}\right)\left(X^{T} y-x_{i} y_{i}\right)$

#### $ω_{-i,λ} =A^{-1} X^{T} y-A^{-1} x_{i} y_{i}+\frac{A^{-1} x_{i} x_{i}^{T} A^{-1}}{1-x_{i}^{T} A^{-1} x_{i}}\left(X^{T} y-x_{i} y_{i}\right)$



#### $step 2$
#### 可以发现 $A^{-1} X^{T} y$ 即为 $ω_λ$, 再定义 $x_i^T A_{-1}x_i = h_i$， 则有

#### $\begin{align}
ω_{-i,λ} & = ω_λ-A^{-1} x_{i} y_{i}+\frac{A^{-1} x_{i} \hat{y}_{i}-A^{-1} x_{i} h_{i} y_{i}}{1-h_{i}} \\
& =ω_λ+\frac{-A^{-1} x_{i} y_{i}+A^{-1} x_{i} h_{i} y_{i}-A^{-1} x_{i} h_{i} y_{i}+A^{-1} x_{i} \hat{y}_{i}}{1-h_{i}} \\
& =ω_i+\frac{A^{-1} x_{i}\left(\hat{y}_{i}-y_{i}\right)}{1-h_{i}}
\end{align}$

#### $\begin{align}
y_{i}-x_{i}^{T} ω_{-i, \lambda} & =y_{i}-x_{i}^{T}\left(ω+\frac{A^{-1} x_{i}}{1-h_{i}}\left(\hat{y}_{i}-y_{i}\right)\right) \\
& =y_{i}-\hat{y}_{i}-\frac{h_{i}}{1-h_{i}}\left(\hat{y}_{i}-y_{i}\right) \\
& =\left(y_{i}-\hat{y}_{i}\right)\left(1+\frac{h_{i}}{1-h_{i}}\right) \\
& =\frac{y_{i}-\hat{y}_{i}}{1-h_{i}}
\end{align}$

#### 故综上可得：
#### $loocv\_loss=\frac{1}{n} \sum_{i=1}^{n}\left|\frac{y_{i}-\hat{y}_{i}}{1-h_{i}}\right|^{2} $
#### $h_{i}=x_{i}^{T} A^{-1} x_{i}, \quad A=X^{T} X+\lambda I$

#### 即对于每次选定的 $λ$ 后没必要训练 $n$ 个模型求损失函数， 可以直接计算该 $λ$ 下的预测值即可直接求出 $loss$

----
- #### sklearn 中模型使用

- ##### 不使用交叉检验的Riage模型： linear_model.Ridge
- ##### 使用交叉检验的Riage模型 linear_model.RidgeCV



In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model
from mpl_toolkits.mplot3d import Axes3D
from sklearn.metrics import mean_squared_error,r2_score

#定义样本和特征数量
num_sample=5000
num_feature=50
weight=-3.4
b_true=4.3
feature=np.random.normal(size=(num_sample,num_feature))
label=weight*feature+b_true+np.random.normal(size=(num_sample,num_feature))

# Split the data into training/testing sets
X_train = feature[:-100,:]
X_test = feature[-100:,:]

# Split the targets into training/testing sets
y_train = label[:-100]
y_test = label[-100:]

#创建Riage模型
reg=linear_model.Ridge(alpha=0.5,fit_intercept=True)
regCV=linear_model.RidgeCV(alphas=[0.1,0.3,0.5,0.7,1, 2, 3, 4, 5],fit_intercept=True)
#传入训练集训练模型
reg.fit(X_train,y_train)
regCV.fit(X_train,y_train)
#传入测试集得到结果
y_predict=reg.predict(X_test)
y_CV_predict=reg.predict(X_test)
#求解均方误差
# print(y_predict, y_CV_predict)
print(mean_squared_error(y_test,y_predict))
print(mean_squared_error(y_test,y_CV_predict))
print('RidgeCV最优的alpha_: ',regCV.alpha_)
# #求解R^2,最大值为1，越接近1，模型越完美
print(r2_score(y_test, y_predict))
print(r2_score(y_test, y_CV_predict))
# print("Coefficient of the model:%.2f"%reg.coef_)
# print("intercept of the model:%.2f"%reg.intercept_)

# ax = plt.subplot(111)
# ax.scatter(X_test,y_test)
# ax.plot(X_test,y_predict)

# plt.show()

# 不知道为什么最优RidgeCV的alpha_与Ridge不同但回归效果相同

1.0264043578139008
1.0264043578139008
RidgeCV最优的alpha_:  4.0
0.9172936847296822
0.9172936847296822


> #### 参考https://zhuanlan.zhihu.com/p/607797639
> #### 参考https://zhuanlan.zhihu.com/p/165493873